In [1]:
from tqdm.auto import tqdm
from datasets import load_dataset
from transformers import AutoTokenizer, GPT2LMHeadModel

In [18]:
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2-large")

model = GPT2LMHeadModel.from_pretrained("/home/drdo/Caricatures/models/scan_gpt2-large/checkpoint-160000")

model.generation_config.max_length = 256

In [4]:
dataset = load_dataset("scan", "simple", trust_remote_code=True)

In [5]:
dataset['test'][4]

{'commands': 'run thrice and walk opposite left',
 'actions': 'I_RUN I_RUN I_RUN I_TURN_LEFT I_TURN_LEFT I_WALK'}

In [10]:
inputs = tokenizer(
    "run thrice and walk opposite left<sep>", return_tensors="pt")
tokenizer.decode(model.generate(**inputs)[0], skip_special_tokens=False)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'I_RUN I_RUN I_RUN I_TURN_LEFT I_TURN_LEFT I_WALK'

In [17]:
test_dataset = dataset['test']#.select(range(100))

In [19]:
count = 0
model.to("cuda")
bar = tqdm(range(len(test_dataset)))
for example in test_dataset:
    command = example['commands']
    label = example['actions']
    inputs = tokenizer(command+" <sep> ", return_tensors="pt").to("cuda")
    output = model.generate(**inputs)[0].to("cpu")
    #output = model.generate(**inputs, num_beams=5, max_new_tokens=256)[0]
    output = tokenizer.decode(output, skip_special_tokens=True).split("<sep>")[-1].strip()
    if output == label:
        count += 1
    bar.update(1)

print(count/len(test_dataset))

  0%|          | 0/4182 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

0.6661884265901482


In [ ]:
# distilgpt2 : acc = 0.6406025824964132
# distilgpt2 : acc = 0.67 -> beam search (beam size = 3)
# gpt2 : acc = 0.810856049736968, 0.88, 